In [3]:
import rasa_core
from rasa_core.agent import Agent
from bokeh.io import output_notebook
agent = Agent.load("models/dialogue_embed")
# agent = Agent.load("models/dialogue_embed_before_norm_loss")
output_notebook()

UnsupportedDialogueModelError: The model version is to old to be loaded by this Rasa Core instance. Either retrain the model, or run withan older version. Model version: 0.13.0a5 Instance version: 0.14.0a1 Minimal compatible version: 0.14.0a1

In [4]:
rasa_core.__version__

'0.14.0a1'

In [5]:
policy = agent.policy_ensemble.policies[0]
interpreter = agent.interpreter
print(policy.sparse_attention)

NameError: name 'agent' is not defined

In [6]:
domain = agent.domain


NameError: name 'agent' is not defined

In [7]:
from rasa_core.training.structures import StoryGraph
from rasa_core.training.dsl import StoryFileReader
from rasa_core.trackers import DialogueStateTracker
from rasa_core.channels import UserMessage
import numpy as np
np.set_printoptions(precision=3, suppress=True)

def embedding_from_tracker(story, bot=False, predict_only=False):
    reader = StoryFileReader(domain, interpreter)
    story = story.split('\n')
    story_steps = reader.process_lines(story)
#     story_graph = StoryGraph(story_steps)
    tracker = DialogueStateTracker(
            1,
            domain.slots
    )
    tracker._reset()
    for step in story_steps:
        events = step.explicit_events(domain)
        for event in events[:-1]:
            tracker.update(event)

    if predict_only:
        return policy.predict_action_probabilities(tracker, domain)

    feed_dict=policy.tf_feed_dict_for_prediction(tracker, domain)

    dial_vec = policy.session.run(policy.dial_embed, feed_dict=feed_dict)
    bot_vec = policy.session.run(policy.bot_embed, feed_dict=feed_dict)

    no_skip_gate = policy.session.run(policy.copy_attn_debug, feed_dict=feed_dict)
    attn_embed = policy.session.run(policy.attn_embed, feed_dict=feed_dict)
    rnn_embed = policy.session.run(policy.rnn_embed, feed_dict=feed_dict)
    probs = policy.session.run(policy.alignment_history, feed_dict=feed_dict)
    if policy.all_time_masks is not None:
        time_mask = policy.session.run(policy.all_time_masks, feed_dict=feed_dict)
    else:
        time_mask = None
    if policy.topics is not None:
        topics = policy.session.run(policy.topics, feed_dict=feed_dict)
    else:
        topics = None
    cell_state = policy.session.run(policy.cell_state, feed_dict=feed_dict)
    if probs.shape[2] == 2 * probs.shape[1]:
        user_probs = probs[0, -1, :int(probs.shape[2]/2)]
        bot_probs = probs[0, -1, int(probs.shape[2]/2):]
    else:
        user_probs = probs[0, -1, :]
        bot_probs = []

    sims = policy.session.run(policy.sim_op, feed_dict=feed_dict)
    if bot is False:
        sims1 = policy.predict_action_probabilities(tracker, domain)

    del tracker
#     print(np.shape(dial_vec))

    if bot is False:
        return (dial_vec[0,-1,:],
                no_skip_gate[0,-1,:],
                attn_embed[0,-1,:],
                rnn_embed[0,-1,:],
                [user_probs, bot_probs, time_mask[0,-1,:] if time_mask is not None else time_mask],
                sims[0,-1,:],
                sims1,
                topics[0] if topics is not None else topics,
                cell_state[0]
               )
    else:
        return (dial_vec[0,-1,:],
                sims[0,-1,:],
                bot_vec[0,-1,:,:])
    
def make_prediction(story):
    dial, sim0, bot = embedding_from_tracker(story, bot=True)
    abs_vals = np.array([np.absolute(vec) for vec in bot])
    norms = np.apply_along_axis(np.linalg.norm, 1, bot)
    sims = np.dot(bot, dial)/norms
#     name = domain.action_for_index(np.argmax(sims)).name()
#     print(name)
    name = domain.action_names[np.argmax(sim0)]
#     print(name)
#     print('---')
#     ids = sim0.argsort()[::-1]
#     print(ids)
#     print(sims[ids])
    return '- ' + name, np.max(sim0)
#     print(dial.shape, bot.shape)

In [8]:
# print(domain.intents)
story_block = """* request_hotel
    - utter_ask_details
* chitchat
    - utter_chitchat
    - utter_ask_details
* inform{"enddate": "May 26th"}
    - slot{"enddate": "May 26th"}
    - utter_ask_startdate
* inform{"startdate": "next week"}
    - slot{"startdate": "next week"}
    - utter_ask_location
* correct{"startdate": "next week"}
    - slot{"startdate": "next week"}
    - utter_correct_startdate_hotel
    - utter_ask_location
* explain
    - utter_explain_location_hotel
    - utter_ask_location
* inform{"location": "rome"}
    - slot{"location": "rome"}
    - utter_ask_price
* chitchat
    - utter_chitchat
    - utter_ask_price
* chitchat
    - utter_chitchat
    - utter_ask_price
* chitchat
    - utter_chitchat
    - utter_ask_price
* chitchat
    - utter_chitchat
    - utter_ask_price
* explain
    - utter_explain_price_hotel
    - utter_ask_price
* inform{"price": "expensive"}
    - utter_ask_people
* inform{"people": "4"}
    - utter_filled_slots
    - action_search_hotel
    - slot{"hotel": "hotel"}
    - utter_suggest_hotel
* chitchat
    - utter_chitchat
    - utter_suggest_hotel
* correct{"people": "2"}
    - slot{"people": "2"}
    - correct: utter_correct_people_hotel
    - correct: action_search_hotel
    - slot{"hotel": "hotel"}
    - correct: utter_suggest_hotel
* affirm
    - utter_happy
* request_restaurant
    - restaurant: utter_ask_details
* inform{"people": "4"}
    - slot{"people": "4"}
    - restaurant: utter_ask_location
* inform{"location": "paris"}
    - slot{"location": "paris"}
    - restaurant: utter_ask_price
* explain
    - explain: utter_explain_price_restaurant
    - explain: utter_ask_price
* did_that_work
    - did_that_work: utter_more_info_restaurant
    - did_that_work: utter_ask_price
* correct{"location": "london"}
    - slot{"location": "london"}
    - correct: utter_correct_location_restaurant
    - correct: utter_ask_price
* inform{"price": "expensive"}
    - restaurant: utter_ask_cuisine
* inform{"cuisine": "italian"}
    - restaurant: utter_filled_slots
    - restaurant: action_search_restaurant
    - slot{"restaurant": "restaurant"}
    - restaurant: utter_suggest_restaurant
* affirm
    - restaurant: utter_happy"""

In [9]:
# story_block = """* request_restaurant
#     - restaurant: utter_ask_details
# * inform{"people": "4"}
#     - slot{"people": "4"}
#     - restaurant: utter_ask_location
# * inform{"location": "paris"}
#     - slot{"location": "paris"}
#     - restaurant: utter_ask_price
# * explain
#     - explain: utter_explain_price_restaurant
#     - explain: utter_ask_price
# * did_that_work
#     - did_that_work: utter_more_info_restaurant
#     - did_that_work: utter_ask_price
# * correct{"location": "london"}
#     - slot{"location": "london"}
#     - correct: utter_correct_location_restaurant
#     - correct: utter_ask_price
# * inform{"price": "expensive"}
#     - restaurant: utter_ask_cuisine
# * inform{"cuisine": "italian"}
#     - restaurant: utter_filled_slots
#     - restaurant: action_search_restaurant
#     - slot{"restaurant": "restaurant"}
#     - restaurant: utter_suggest_restaurant
# * affirm
#     - restaurant: utter_happy"""

In [10]:
# story_block = """* request_hotel
#     - utter_ask_details
#     - slot{"hotel": null}
# * inform{"price": "mid-range"}
#     - slot{"price": "mid-range"}
#     - slot{"price": "mid-range"}
#     - utter_ask_people
# * correct{"price": "cheap"}
#     - slot{"price": "cheap"}
#     - slot{"price": "cheap"}
#     - utter_correct_price_hotel
#     - utter_ask_people
# * explain
#     - utter_explain_people_hotel
#     - utter_ask_people
# * explain
#     - utter_explain_people_hotel
#     - utter_ask_people
# * inform{"people": "2"}
#     - slot{"people": "2"}
#     - slot{"people": "2"}
#     - utter_ask_startdate
# * did_that_work
#     - utter_more_info_hotel
#     - utter_ask_startdate
# * explain
#     - utter_explain_startdate_hotel
#     - utter_ask_startdate
# * correct{"people": "4"}
#     - slot{"people": "4"}
#     - slot{"people": "4"}
#     - utter_correct_people_hotel
#     - utter_ask_startdate
# * inform{"startdate": "10.03.2018"}
#     - slot{"startdate": "10.03.2018"}
#     - utter_ask_enddate
# * inform{"enddate": "10.03.2018"}
#     - slot{"enddate": "10.03.2018"}
#     - utter_ask_location   <!-- predicted: utter_explain_location_hotel -->
# * did_that_work
#     - utter_more_info_hotel
#     - utter_ask_location
# * chitchat
#     - utter_chitchat
#     - utter_ask_location
# * inform{"location": "paris"}
#     - slot{"location": "paris"}
#     - utter_filled_slots
#     - action_search_hotel
#     - slot{"hotel": "hotel"}
#     - utter_suggest_hotel
# * did_that_work
#     - utter_worked_hotel
#     - utter_suggest_hotel
# * affirm
#     - utter_happy"""

In [11]:
print(len(story_block.split('\n')))
cool_list = []
lines = story_block.split('\n')
for idx, line in enumerate(lines):
    if idx == len(lines)-1:
        continue
    if '*' in lines[idx+1] or 'slot{' in lines[idx+1]:
#     if 'slot{' in lines[idx+1]:
        continue
    sub_story = '\n'.join(lines[:idx+1])
    (dial, not_skip, attn, rnn, probs, sims, sims1, topic, cell_state
    ) = embedding_from_tracker("#generate story\n{}".format(sub_story))
#     print(np.max(sims), np.max(sims1))

    pred, sim = make_prediction("#generate story\n{}".format(sub_story))
    cool_list.append((lines[idx+1].strip(), dial, (pred, sim, sims), not_skip, attn[-dial.shape[-1]:], rnn,
                      probs, topic, cell_state))
#     embedding_from_tracker("#generate story\n{}".format(sub_story), predict_only=True)

print(len(cool_list))
# print(cool_list[0][4].shape)
#     break
# dial = embedding_from_tracker("#generate story\n{}".format(story_block)
# )
# print(dial.shape)
# print(len(story_block.split('\n')))
# print(probs[0, 6, :])
print(dial.shape[-1])

81


NameError: name 'domain' is not defined

In [12]:
import copy

# cool_list0 = copy.deepcopy(cool_list)
cool_list[7]

IndexError: list index out of range

In [13]:
# cool_list1= copy.deepcopy(cool_list)
cool_list[-1]

IndexError: list index out of range

In [14]:
# import matplotlib.pyplot as plt
# plt.plot(cool_list[15][-2], marker='x')

In [15]:
# plt.plot(cool_list[19][-2], marker='x')

In [16]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
X = pca.fit_transform([ting[1] for ting in cool_list])

In [ ]:
from bokeh.plotting import figure, show
import matplotlib.pyplot as plt

In [ ]:
embed_dim = cool_list[0][1].shape[0]


In [ ]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool, Arrow, OpenHead

hover = HoverTool(tooltips=[("true action","@desc"), ("prediction", "@pred"),
                            ("not_skip_gate", "@not_skip")],names=['circle'])
p = figure( tools=[hover])

for i in range(1, len(X)):
    p.add_layout(Arrow(end=OpenHead(line_color="firebrick", size=10, line_width=2),
                       x_start=X[i-1,0], y_start=X[i-1,1],
                       x_end=X[i,0], y_end=X[i,1], line_color='gray'))

source = ColumnDataSource(data=dict(
    x=X[:,0],
    y=X[:,1],
    desc=[ting[0] for ting in cool_list],
    pred = [ting[2][0] for ting in cool_list],
    not_skip = [ting[3][-1] for ting in cool_list]
))
p.circle('x','y', source=source, size=10, name='circle')
# p.line(X[:,0], X[:,1])

X1 = pca.transform([ting[4] for ting in cool_list])
source = ColumnDataSource(data=dict(
    x=X1[:,0],
    y=X1[:,1],
    desc=[ting[0] for ting in cool_list],
    pred = [ting[2][0] for ting in cool_list],
    not_skip = [ting[3][-1] for ting in cool_list]
))
p.circle('x','y', source=source, size=5, color='green', name='circle')

X2 = pca.transform([ting[5] for ting in cool_list])
source = ColumnDataSource(data=dict(
    x=X2[:,0],
    y=X2[:,1],
    desc=[ting[0] for ting in cool_list],
    pred = [ting[2][0] for ting in cool_list],
    not_skip = [ting[3][-1] for ting in cool_list]
))
p.circle('x','y', source=source, size=5, color='red', name='circle')

x = pca.transform(np.zeros((1,embed_dim)))
p.circle(x[0,0], x[0,1], size=5, color='magenta')
p.circle(X[0,0], X[0,1], size=10, color='green')
p.circle(X[-1,0], X[-1,1], size=10, color='red')


In [2]:
show(p)

NameError: name 'show' is not defined

In [18]:
print('utter_ask_price', '-->', domain.index_for_action('utter_ask_price'))
print('utter_ask_details', '-->', domain.index_for_action('utter_ask_details'))
print('utter_ask_people', '-->', domain.index_for_action('utter_ask_people'))
print('utter_explain_details_hotel', '-->', domain.index_for_action('utter_explain_details_hotel'))
print('utter_explain_people_hotel', '-->', domain.index_for_action('utter_explain_people_hotel'))
print('utter_explain_price_hotel', '-->', domain.index_for_action('utter_explain_price_hotel'))
print('utter_ask_location', '-->', domain.index_for_action('utter_ask_location'))
print('utter_explain_location_hotel', '-->', domain.index_for_action('utter_explain_location_hotel'))
print('utter_ask_startdate', '-->', domain.index_for_action('utter_ask_startdate'))
print('utter_suggest_hotel', '-->', domain.index_for_action('utter_suggest_hotel'))
print('action_search_hotel', '-->', domain.index_for_action('action_search_hotel'))
print('action_listen', '-->', domain.index_for_action('action_listen'))

utter_ask_price --> 6
utter_ask_details --> 28
utter_ask_people --> 4
utter_explain_details_hotel --> 14
utter_explain_people_hotel --> 19
utter_explain_price_hotel --> 21
utter_ask_location --> 5
utter_explain_location_hotel --> 17
utter_ask_startdate --> 7
utter_suggest_hotel --> 12
action_search_hotel --> 42
action_listen --> 0


In [19]:
utter_price = embedding_from_tracker("#generate story\n{}".format(story_block), bot=True)[0,-1,5,:]
utter_price.shape

(20,)

In [20]:
print(utter_price.shape)
x = pca.transform(utter_price.reshape(1,-1))
p.circle(x[0][0], x[0][1], color='black')

(20,)


GlyphRenderer(id='1db745d9-cfee-44a7-8a51-a238a358f252', ...)

In [21]:
show(p)

In [22]:
from collections import namedtuple

In [23]:
Point = namedtuple('Point', ['x', 'y'])
p = Point(11, y=22)
p._asdict()

OrderedDict([('x', 11), ('y', 22)])

In [24]:
Point3D = namedtuple('Point3D', Point._fields + ('z',))

In [25]:
p_asdict = p._asdict()
p_asdict['z'] = 3
p_asdict

OrderedDict([('x', 11), ('y', 22), ('z', 3)])

In [26]:
p3D = Point3D(**p_asdict)
p3D

Point3D(x=11, y=22, z=3)

In [27]:
x ={
        "day": "20",
        "month": "06",
        "year": "2018"
    }

In [28]:
x

{'day': '20', 'month': '06', 'year': '2018'}

In [29]:
def value_touched_dict(value):
    if isinstance(value, dict) and value.get("day"):
        value["value1"] = value.pop("day")
        value["value2"] = value.pop("month")
        value["value3"] = value.pop("year")
    return {"value": value,
            "touched": True}
value_touched_dict(x)

{'touched': True, 'value': {'value1': '20', 'value2': '06', 'value3': '2018'}}

In [30]:
value_touched_dict('x')

{'touched': True, 'value': 'x'}